In [6]:
# Import essential libraries
import os
import json
import hashlib
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
import warnings

import numpy as np
import pandas as pd
import yaml
from IPython.display import display

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [7]:
# --- Paths & config ---
ROOT = Path("..").resolve()
CONFIG_PATH = ROOT / "configs" / "config.yaml"

if CONFIG_PATH.exists():
    with open(CONFIG_PATH, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)
else:
    config = {}

DATA_RAW = ROOT / config.get("data", {}).get("raw_path", "Data/")
DATA_PROCESSED = ROOT / config.get("data", {}).get("processed_path", "Data/processed/")

OUTPUTS_DIR = ROOT / "outputs"
EMB_DIR = OUTPUTS_DIR / "embeddings"
VDB_DIR = OUTPUTS_DIR / "vector_db"
IMG_CACHE_DIR = OUTPUTS_DIR / "image_cache"

for p in [OUTPUTS_DIR, EMB_DIR, VDB_DIR, IMG_CACHE_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("📁 Root:", ROOT)
print("📁 Raw data:", DATA_RAW)
print("📁 Processed data:", DATA_PROCESSED)
print("📁 Outputs:", OUTPUTS_DIR)

📁 Root: /home/roshan/Aria/mlops
📁 Raw data: /home/roshan/Aria/mlops/Data
📁 Processed data: /home/roshan/Aria/mlops/Data/processed
📁 Outputs: /home/roshan/Aria/mlops/outputs


In [14]:
# ===== Text Processing Utilities =====

def ensure_string(x: Any) -> str:
    """Convert any value to string, handling None and NaN."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    return str(x).strip()


def normalize_persian_text(text: str) -> str:
    """
    Normalize Persian/Arabic text for consistent processing.
    - Converts Arabic variants (ي, ك) to Persian (ی, ک)
    - Removes extra whitespace
    - Ensures deterministic output
    """
    text = ensure_string(text)
    if not text:
        return ""
    
    # Normalize Arabic to Persian characters
    text = text.replace("ي", "ی").replace("ك", "ک")
    
    # Normalize whitespace
    text = " ".join(text.split())
    
    return text


def build_product_text(row: pd.Series) -> str:
    """
    Build comprehensive product text for embedding.
    Combines name, brand, and category into single text.
    Skips unidentified/noise categories and brands (empty or "شناسایی نشده").
    Note: extra_features are NOT included as they don't provide useful semantic information.
    """
    name = normalize_persian_text(row.get("persian_name", ""))
    brand = normalize_persian_text(row.get("brand_title", ""))
    category = normalize_persian_text(row.get("category_title", ""))
    
    # Filter out noise/unidentified labels
    noise_keywords = ["شناسایی نشده", "not detected", "unknown", "n/a"]
    
    if any(keyword in brand.lower() for keyword in noise_keywords):
        brand = ""
    if any(keyword in category.lower() for keyword in noise_keywords):
        category = ""
    
    # Build parts list with only meaningful values
    parts = []
    if name:
        parts.append(name)
    if brand:
        parts.append(f"برند: {brand}")
    if category:
        parts.append(f"دسته: {category}")
    
    return " | ".join(parts)


print("✅ Text processing utilities ready")

✅ Text processing utilities ready


In [13]:
# Load cleaned base products
BASE_PRODUCTS_PATH = DATA_PROCESSED / "base_products_clean.parquet"
BRANDS_PATH = DATA_PROCESSED / "brands_clean.parquet"
CATEGORIES_PATH = DATA_PROCESSED / "categories_clean.parquet"

if not BASE_PRODUCTS_PATH.exists():
    raise FileNotFoundError(
        f"❌ Cleaned data not found: {BASE_PRODUCTS_PATH}\n"
        "Please run notebook 02 (Data Cleaning) first to generate processed files."
    )

print("📂 Loading cleaned datasets...")
base_products = pd.read_parquet(BASE_PRODUCTS_PATH)
print(f"✅ Loaded base_products: {base_products.shape}")

# Load reference tables
brands = pd.read_parquet(BRANDS_PATH) if BRANDS_PATH.exists() else pd.read_parquet(DATA_RAW / "brands.parquet")
categories = pd.read_parquet(CATEGORIES_PATH) if CATEGORIES_PATH.exists() else pd.read_parquet(DATA_RAW / "categories.parquet")

print(f"✅ Loaded brands: {brands.shape}")
print(f"✅ Loaded categories: {categories.shape}")

# Create mapping dictionaries
brands_map = brands[["id", "title"]].drop_duplicates(subset=["id"]).rename(
    columns={"id": "brand_id", "title": "brand_title"}
)
categories_map = categories[["id", "title"]].drop_duplicates(subset=["id"]).rename(
    columns={"id": "category_id", "title": "category_title"}
)

# Enrich products with readable titles
if "brand_id" in base_products.columns:
    base_products = base_products.merge(brands_map, on="brand_id", how="left")
    print("✅ Enriched with brand titles")

if "category_id" in base_products.columns:
    base_products = base_products.merge(categories_map, on="category_id", how="left")
    print("✅ Enriched with category titles")

# Filter out unidentified/noise categories and brands
print("\n🧹 Filtering unidentified categories and brands...")

# Replace unidentified categories (id=-1) with empty string
if "category_id" in base_products.columns:
    unidentified_cat_mask = base_products["category_id"] == -1
    if unidentified_cat_mask.any():
        base_products.loc[unidentified_cat_mask, "category_title"] = ""
        print(f"   ✓ Filtered {unidentified_cat_mask.sum():,} products with unidentified category (id=-1)")

# Replace unidentified brands (id=-1) with empty string  
if "brand_id" in base_products.columns:
    unidentified_brand_mask = base_products["brand_id"] == -1
    if unidentified_brand_mask.any():
        base_products.loc[unidentified_brand_mask, "brand_title"] = ""
        print(f"   ✓ Filtered {unidentified_brand_mask.sum():,} products with unidentified brand (id=-1)")

# Display sample
print("\n📋 Sample enriched products:")
display(base_products[["random_key", "persian_name", "brand_title", "category_title"]].head(3))

📂 Loading cleaned datasets...
✅ Loaded base_products: (1022296, 8)
✅ Loaded brands: (2025, 2)
✅ Loaded categories: (746, 3)
✅ Enriched with brand titles
✅ Enriched with category titles

🧹 Filtering unidentified categories and brands...
   ✓ Filtered 844,870 products with unidentified brand (id=-1)

📋 Sample enriched products:


,random_key,persian_name,brand_title,category_title
0,chjknu,فرش 700شانه اپال فیروزه ای,,فرش ماشینی
1,ppndkv,صندلی تاب مدل chioco,,سایر صندلی
2,lwtjlj,بشقاب مجموعه کیک یکپارچهسازی با سیستمعامل 13 ق...,انگلیش هوم / English Home,دیس و بشقاب


In [11]:
# Import text embedding dependencies

import torch
from sentence_transformers import SentenceTransformer
    

# Model configuration
TEXT_MODEL_NAME = config.get("feature_engineering", {}).get("text", {}).get(
    "model_name", "abbas"
)
TEXT_BATCH_SIZE = config.get("feature_engineering", {}).get("text", {}).get(
    "batch_size", 64
)

# Device selection
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

print(f"\n🤖 Text Embedding Configuration:")
print(f"   Model: {TEXT_MODEL_NAME}")
print(f"   Device: {DEVICE}")
print(f"   Batch size: {TEXT_BATCH_SIZE}")

# Load model
print("\n⏳ Loading text model...")
text_model = SentenceTransformer(TEXT_MODEL_NAME, device=DEVICE)
print("✅ Text model loaded successfully")

cuda

🤖 Text Embedding Configuration:
   Model: sentence-transformers/all-MiniLM-L12-v2
   Device: cuda
   Batch size: 64

⏳ Loading text model...
✅ Text model loaded successfully


In [19]:
# Prepare working dataset
# For demonstration, use a sample. For production, process full dataset in batches.
SAMPLE_SIZE = None  # Set to None for full dataset

print(f"\n📋 Preparing working dataset (sample={SAMPLE_SIZE})...")

# Select relevant columns (excluding extra_features - not useful for embeddings)
cols = ["random_key", "persian_name"]
for optional_col in ["brand_title", "category_title", "image_url"]:
    if optional_col in base_products.columns:
        cols.append(optional_col)

work_df = base_products[cols].copy()

# Sample if needed
if SAMPLE_SIZE is not None and len(work_df) > SAMPLE_SIZE:
    work_df = work_df.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED)
    print(f"✅ Sampled {SAMPLE_SIZE} products from {len(base_products):,}")
else:
    print(f"✅ Using all {len(work_df):,} products")

work_df = work_df.reset_index(drop=True)

# Build product text for each row
print("⏳ Building product texts...")
work_df["product_text"] = work_df.apply(build_product_text, axis=1)

print(f"✅ Product texts created: {len(work_df):,}")
print(f"\n📝 Example product text:")
print(work_df["product_text"].iloc[1][:1000] + "...")


📋 Preparing working dataset (sample=None)...
✅ Using all 1,022,296 products
⏳ Building product texts...
✅ Product texts created: 1,022,296

📝 Example product text:
صندلی تاب مدل chioco | دسته: سایر صندلی...


In [20]:
# Generate text embeddings
print("\n⏳ Generating text embeddings...")

# MiniLM doesn't use prefixes - use text directly
texts = work_df["product_text"].tolist()

# Encode with normalization
text_embeddings = text_model.encode(
    texts,
    batch_size=TEXT_BATCH_SIZE,
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True
)

text_embeddings = np.asarray(text_embeddings, dtype=np.float32)

print(f"✅ Text embeddings generated:")
print(f"   Shape: {text_embeddings.shape}")
print(f"   Dtype: {text_embeddings.dtype}")
print(f"   Sample norm: {float(np.linalg.norm(text_embeddings[0])):.4f}")


⏳ Generating text embeddings...


Batches: 100%|██████████| 15974/15974 [05:03<00:00, 52.70it/s]


✅ Text embeddings generated:
   Shape: (1022296, 384)
   Dtype: float32
   Sample norm: 1.0000


In [23]:
# Stratified sampling for images to preserve category distribution
MAX_IMAGES = 10000  # Maximum number of images to download and embed

if "image_url" in work_df.columns and "category_title" in work_df.columns:
    print(f"\n📊 Stratified Image Sampling (preserving category distribution)...")
    print(f"   Target: {MAX_IMAGES:,} images")
    
    # Filter products that have valid image URLs
    products_with_images = work_df[work_df["image_url"].notna() & (work_df["image_url"] != "")].copy()
    
    print(f"   Products with images: {len(products_with_images):,}")
    
    if len(products_with_images) > MAX_IMAGES:
        from sklearn.model_selection import train_test_split
        
        # Use category_title for stratification (handle missing categories)
        products_with_images["_strata"] = products_with_images["category_title"].fillna("_unknown")
        
        # Count samples per category
        category_counts = products_with_images["_strata"].value_counts()
        
        # Separate categories with only 1 sample (can't stratify these)
        single_sample_categories = category_counts[category_counts == 1].index.tolist()
        multi_sample_mask = ~products_with_images["_strata"].isin(single_sample_categories)
        
        products_multi = products_with_images[multi_sample_mask].copy()
        products_single = products_with_images[~multi_sample_mask].copy()
        
        print(f"   Categories with ≥2 samples: {len(products_multi):,} products")
        print(f"   Categories with 1 sample: {len(products_single):,} products")
        
        # Calculate how many to sample from multi-sample categories
        n_from_multi = min(MAX_IMAGES, len(products_multi))
        
        if len(products_multi) > 0 and n_from_multi > 0:
            # Stratified sample from multi-sample categories
            sampled_multi, _ = train_test_split(
                products_multi,
                train_size=n_from_multi,
                stratify=products_multi["_strata"],
                random_state=RANDOM_SEED
            )
        else:
            sampled_multi = pd.DataFrame()
        
        # Add single-sample categories if we have room
        remaining_slots = MAX_IMAGES - len(sampled_multi)
        if remaining_slots > 0 and len(products_single) > 0:
            n_single = min(remaining_slots, len(products_single))
            sampled_single = products_single.sample(n=n_single, random_state=RANDOM_SEED)
            sampled_images = pd.concat([sampled_multi, sampled_single], ignore_index=False)
        else:
            sampled_images = sampled_multi
        
        sampled_images = sampled_images.drop(columns=["_strata"])
        
        print(f"   ✅ Sampled {len(sampled_images):,} images with stratification")
        
        # Show category distribution
        print(f"\n   📈 Category Distribution (Top 10):")
        category_dist = sampled_images["category_title"].value_counts().head(10)
        for cat, count in category_dist.items():
            cat_display = cat if cat else "(no category)"
            pct = (count / len(sampled_images)) * 100
            print(f"      {cat_display}: {count:,} ({pct:.1f}%)")
        
    else:
        sampled_images = products_with_images
        print(f"   ✅ Using all {len(sampled_images):,} products with images (less than {MAX_IMAGES:,})")
    
    # Store the indices of products selected for image embedding
    image_sample_indices = sampled_images.index.tolist()
    
else:
    print("⚠️  Skipping image sampling - no image_url or category_title column")
    image_sample_indices = []


📊 Stratified Image Sampling (preserving category distribution)...
   Target: 10,000 images
   Products with images: 1,018,955
   Categories with ≥2 samples: 1,018,947 products
   Categories with 1 sample: 8 products
   ✅ Sampled 10,000 images with stratification

   📈 Category Distribution (Top 10):
      تابلو و مجسمه: 378 (3.8%)
      فرش ماشینی: 281 (2.8%)
      رومیزی و پرده: 204 (2.0%)
      انواع پارچه: 185 (1.8%)
      روتختی، لحاف و سرویس خواب: 179 (1.8%)
      ماگ: 178 (1.8%)
      اشیاء قدیمی و کلکسیونی: 146 (1.5%)
      ساعت دیواری٬ رومیزی و تزئینی: 145 (1.5%)
      لوازم یدکی آشپزخانه: 140 (1.4%)
      پارچ و لیوان: 137 (1.4%)


In [21]:
# Import image embedding dependencies

import requests
from PIL import Image
from transformers import CLIPModel, CLIPProcessor


# Model configuration
IMAGE_MODEL_NAME = config.get("feature_engineering", {}).get("image", {}).get(
    "model_name", "openai/clip-vit-base-patch32"
)

print(f"\n🖼️  Image Embedding Configuration:")
print(f"   Model: {IMAGE_MODEL_NAME}")
print(f"   Device: {DEVICE}")
print(f"   Cache directory: {IMG_CACHE_DIR}")

# Load CLIP model and processor
print("\n⏳ Loading CLIP model...")
clip_model = CLIPModel.from_pretrained(IMAGE_MODEL_NAME).to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained(IMAGE_MODEL_NAME)
clip_model.eval()
print("✅ CLIP model loaded successfully")


🖼️  Image Embedding Configuration:
   Model: openai/clip-vit-base-patch32
   Device: cuda
   Cache directory: /home/roshan/Aria/mlops/outputs/image_cache

⏳ Loading CLIP model...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


✅ CLIP model loaded successfully


In [24]:
# Image processing utilities

def fetch_image(url: str, timeout: int = 15) -> Optional[Image.Image]:
    """
    Download and cache an image from URL.
    Returns RGB PIL Image or None if failed.
    """
    url = ensure_string(url)
    if not url:
        return None
    
    # Create cache filename from URL hash
    cache_filename = hashlib.sha256(url.encode()).hexdigest() + ".jpg"
    cache_path = IMG_CACHE_DIR / cache_filename
    
    try:
        # Check cache first
        if cache_path.exists() and cache_path.stat().st_size > 0:
            img = Image.open(cache_path)
            return img.convert("RGB")
        
        # Download image
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        
        # Save to cache
        with open(cache_path, "wb") as f:
            f.write(response.content)
        
        # Load and return
        img = Image.open(cache_path)
        return img.convert("RGB")
    
    except Exception:
        return None


@torch.inference_mode()
def embed_images_batch(urls: List[str], batch_size: int = 32) -> Tuple[np.ndarray, List[bool]]:
    """
    Generate CLIP embeddings for a list of image URLs.
    
    Returns:
        embeddings: numpy array of embeddings (only for successful images)
        success_mask: boolean list indicating which URLs were successfully processed
    """
    embeddings_list = []
    success_mask = []
    
    # Process in batches
    for i in range(0, len(urls), batch_size):
        batch_urls = urls[i:i + batch_size]
        batch_images = []
        batch_success = []
        
        # Fetch images for this batch
        for url in batch_urls:
            img = fetch_image(url)
            if img is not None:
                batch_images.append(img)
                batch_success.append(True)
            else:
                batch_success.append(False)
        
        # Process valid images
        if batch_images:
            inputs = clip_processor(images=batch_images, return_tensors="pt")
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            
            features = clip_model.get_image_features(**inputs)
            features = torch.nn.functional.normalize(features, p=2, dim=1)
            features_np = features.cpu().numpy().astype(np.float32)
            
            embeddings_list.append(features_np)
        
        success_mask.extend(batch_success)
    
    if not embeddings_list:
        return np.zeros((0, 512), dtype=np.float32), success_mask
    
    return np.vstack(embeddings_list), success_mask


print("✅ Image processing utilities ready")

✅ Image processing utilities ready


In [25]:
# Generate image embeddings for stratified sample only
if "image_url" not in work_df.columns or len(image_sample_indices) == 0:
    print("⚠️  No images to embed")
    image_embeddings = np.zeros((0, 512), dtype=np.float32)
    image_success_mask = [False] * len(work_df)
    image_product_indices = []
else:
    print(f"\n⏳ Generating image embeddings for {len(image_sample_indices):,} stratified products...")
    
    # Get URLs only for sampled products
    sampled_urls = [work_df.loc[idx, "image_url"] for idx in image_sample_indices]
    
    image_embeddings, success_mask_sampled = embed_images_batch(sampled_urls, batch_size=32)
    
    # Map success mask back to full work_df indices
    image_success_mask = [False] * len(work_df)
    image_product_indices = []
    
    for i, idx in enumerate(image_sample_indices):
        if success_mask_sampled[i]:
            image_success_mask[idx] = True
            image_product_indices.append(idx)
    
    print(f"✅ Image embeddings generated:")
    print(f"   Shape: {image_embeddings.shape}")
    print(f"   Successfully embedded: {len(image_product_indices)} / {len(image_sample_indices)}")
    print(f"   Success rate: {len(image_product_indices) / len(image_sample_indices) * 100:.1f}%")


⏳ Generating image embeddings for 10,000 stratified products...
✅ Image embeddings generated:
   Shape: (9982, 512)
   Successfully embedded: 9982 / 10000
   Success rate: 99.8%


In [29]:
# Import vector database

import chromadb
from chromadb.config import Settings
print("✅ ChromaDB available")


# Initialize persistent ChromaDB client
print(f"\n🗄️  Initializing ChromaDB at: {VDB_DIR}")
chroma_client = chromadb.PersistentClient(
    path=str(VDB_DIR),
    settings=Settings(anonymized_telemetry=False)
)

# Metadata for version tracking
run_metadata = {
    "text_model": TEXT_MODEL_NAME,
    "image_model": IMAGE_MODEL_NAME,
    "random_seed": RANDOM_SEED,
    "device": DEVICE,
}
print(run_metadata)
# Get or create collections
try:
    col_text = chroma_client.get_collection("products_text")
    print("✅ Existing text collection loaded")
except:
    col_text = chroma_client.create_collection("products_text", metadata=run_metadata)
    print("✅ New text collection created")

try:
    col_image = chroma_client.get_collection("products_image")
    print("✅ Existing image collection loaded")
except:
    col_image = chroma_client.create_collection("products_image", metadata=run_metadata)
    print("✅ New image collection created")

✅ ChromaDB available

🗄️  Initializing ChromaDB at: /home/roshan/Aria/mlops/outputs/vector_db
{'text_model': 'sentence-transformers/all-MiniLM-L12-v2', 'image_model': 'openai/clip-vit-base-patch32', 'random_seed': 42, 'device': 'cuda'}
✅ New text collection created
✅ New image collection created


In [33]:
# Store text embeddings in ChromaDB
print("\n⏳ Storing text embeddings in ChromaDB...")

ids = work_df["random_key"].astype(str).tolist()
documents = work_df["product_text"].astype(str).tolist()

metadatas = []
for _, row in work_df.iterrows():
    metadatas.append({
        "modality": "text",
        "persian_name": ensure_string(row.get("persian_name", "")),
        "brand_title": ensure_string(row.get("brand_title", "")),
        "category_title": ensure_string(row.get("category_title", "")),
    })

# Batch upsert to avoid exceeding ChromaDB's max batch size
BATCH_SIZE = 5461
total_stored = 0

for i in range(0, len(ids), BATCH_SIZE):
    batch_ids = ids[i:i + BATCH_SIZE]
    batch_embeddings = text_embeddings[i:i + BATCH_SIZE].tolist()
    batch_documents = documents[i:i + BATCH_SIZE]
    batch_metadatas = metadatas[i:i + BATCH_SIZE]
    
    col_text.upsert(
        ids=batch_ids,
        embeddings=batch_embeddings,
        documents=batch_documents,
        metadatas=batch_metadatas
    )
    
    total_stored += len(batch_ids)
    print(f"   Progress: {total_stored:,} / {len(ids):,} text embeddings stored")

print(f"✅ Stored {len(ids):,} text embeddings in ChromaDB")


⏳ Storing text embeddings in ChromaDB...
   Progress: 5,461 / 1,022,296 text embeddings stored
   Progress: 10,922 / 1,022,296 text embeddings stored
   Progress: 16,383 / 1,022,296 text embeddings stored
   Progress: 21,844 / 1,022,296 text embeddings stored
   Progress: 27,305 / 1,022,296 text embeddings stored
   Progress: 32,766 / 1,022,296 text embeddings stored
   Progress: 38,227 / 1,022,296 text embeddings stored
   Progress: 43,688 / 1,022,296 text embeddings stored
   Progress: 49,149 / 1,022,296 text embeddings stored
   Progress: 54,610 / 1,022,296 text embeddings stored
   Progress: 60,071 / 1,022,296 text embeddings stored
   Progress: 65,532 / 1,022,296 text embeddings stored
   Progress: 70,993 / 1,022,296 text embeddings stored
   Progress: 76,454 / 1,022,296 text embeddings stored
   Progress: 81,915 / 1,022,296 text embeddings stored
   Progress: 87,376 / 1,022,296 text embeddings stored
   Progress: 92,837 / 1,022,296 text embeddings stored
   Progress: 98,298 / 1,

In [34]:
# Store image embeddings in ChromaDB
if "image_url" in work_df.columns and len(image_product_indices) > 0:
    print("\n⏳ Storing image embeddings in ChromaDB...")
    
    # Get IDs and URLs for successfully embedded images
    ids = [work_df.loc[idx, "random_key"] for idx in image_product_indices]
    urls = [work_df.loc[idx, "image_url"] for idx in image_product_indices]
    
    metadatas = [{"modality": "image", "image_url": ensure_string(url)} for url in urls]
    
    # Batch upsert to avoid exceeding ChromaDB's max batch size
    BATCH_SIZE = 1000
    total_stored = 0
    
    for i in range(0, len(ids), BATCH_SIZE):
        batch_ids = [str(id_) for id_ in ids[i:i + BATCH_SIZE]]
        batch_embeddings = image_embeddings[i:i + BATCH_SIZE].tolist()
        batch_metadatas = metadatas[i:i + BATCH_SIZE]
        
        col_image.upsert(
            ids=batch_ids,
            embeddings=batch_embeddings,
            metadatas=batch_metadatas
        )
        
        total_stored += len(batch_ids)
        print(f"   Progress: {total_stored:,} / {len(ids):,} image embeddings stored")
    
    print(f"✅ Stored {len(ids):,} image embeddings in ChromaDB")
else:
    print("⚠️  No images to store in ChromaDB")


⏳ Storing image embeddings in ChromaDB...
   Progress: 1,000 / 9,982 image embeddings stored
   Progress: 2,000 / 9,982 image embeddings stored
   Progress: 3,000 / 9,982 image embeddings stored
   Progress: 4,000 / 9,982 image embeddings stored
   Progress: 5,000 / 9,982 image embeddings stored
   Progress: 6,000 / 9,982 image embeddings stored
   Progress: 7,000 / 9,982 image embeddings stored
   Progress: 8,000 / 9,982 image embeddings stored
   Progress: 9,000 / 9,982 image embeddings stored
   Progress: 9,982 / 9,982 image embeddings stored
✅ Stored 9,982 image embeddings in ChromaDB


In [35]:
# Collect package versions
print("\n📝 Collecting metadata...")

package_versions = {}
for package_name in ["torch", "transformers", "sentence_transformers", "chromadb", "numpy", "pandas", "sklearn"]:
    try:
        module = __import__(package_name)
        package_versions[package_name] = module.__version__
    except:
        package_versions[package_name] = "not installed"

# Create comprehensive metadata
metadata = {
    "created_at": pd.Timestamp.now().isoformat(),
    "random_seed": RANDOM_SEED,
    "models": {
        "text_model": TEXT_MODEL_NAME,
        "image_model": IMAGE_MODEL_NAME,
    },
    "data": {
        "total_products": len(base_products),
        "sample_size": SAMPLE_SIZE,
        "processed_products": len(work_df),
        "text_embeddings": len(text_embeddings),
        "max_images_target": MAX_IMAGES if 'MAX_IMAGES' in locals() else None,
        "images_sampled": len(image_sample_indices) if 'image_sample_indices' in locals() else 0,
        "image_embeddings": len(image_product_indices) if 'image_product_indices' in locals() else 0,
    },
    "configuration": {
        "text_batch_size": TEXT_BATCH_SIZE,
        "device": DEVICE,
        "stratified_sampling": True,
    },
    "package_versions": package_versions,
}

# Save metadata
metadata_path = EMB_DIR / "feature_engineering_metadata.json"
with open(metadata_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"✅ Metadata saved to: {metadata_path}")
print("\n" + "="*60)
print("FEATURE ENGINEERING METADATA")
print("="*60)
print(json.dumps(metadata, ensure_ascii=False, indent=2))


📝 Collecting metadata...
✅ Metadata saved to: /home/roshan/Aria/mlops/outputs/embeddings/feature_engineering_metadata.json

FEATURE ENGINEERING METADATA
{
  "created_at": "2026-01-04T14:29:38.606704",
  "random_seed": 42,
  "models": {
    "text_model": "sentence-transformers/all-MiniLM-L12-v2",
    "image_model": "openai/clip-vit-base-patch32"
  },
  "data": {
    "total_products": 1022296,
    "sample_size": null,
    "processed_products": 1022296,
    "text_embeddings": 1022296,
    "max_images_target": 10000,
    "images_sampled": 10000,
    "image_embeddings": 9982
  },
  "configuration": {
    "text_batch_size": 64,
    "device": "cuda",
    "stratified_sampling": true
  },
  "package_versions": {
    "torch": "2.8.0+cu128",
    "transformers": "4.56.1",
    "sentence_transformers": "4.0.2",
    "chromadb": "1.4.0",
    "numpy": "2.2.6",
    "pandas": "2.3.2",
    "sklearn": "1.6.1"
  }
}


In [42]:
# Test text retrieval with sample queries
print("\n" + "="*60)
print("RETRIEVAL VALIDATION TEST")
print("="*60)

def embed_query(query: str) -> np.ndarray:
    """Embed a user query (MiniLM doesn't use prefixes)."""
    normalized_query = normalize_persian_text(query)
    embedding = text_model.encode([normalized_query], normalize_embeddings=True)
    return np.asarray(embedding, dtype=np.float32)


# Test with sample Persian queries
test_queries = [
    "یخچال",
    "یخچال فریزر طرح ساید پاکشوما",
    "بطری شیشه ای"
]

for query in test_queries:
    print(f"\n🔍 Query: {query}")
    print("-" * 60)
    
    query_embedding = embed_query(query)
    
    results = col_text.query(
        query_embeddings=query_embedding.tolist(),
        n_results=3,
        include=["documents", "distances", "metadatas"]
    )
    
    for i, (doc_id, distance, doc, metadata) in enumerate(zip(
        results["ids"][0],
        results["distances"][0],
        results["documents"][0],
        results["metadatas"][0]
    )):
        print(f"\n{i+1}. Product ID: {doc_id}")
        print(f"   Distance: {distance:.4f}")
        print(f"   Brand: {metadata.get('brand_title', 'N/A')}")
        print(f"   Category: {metadata.get('category_title', 'N/A')}")
        print(f"   Text: {doc[:150]}...")

print("\n" + "="*60)
print("✅ Feature Engineering Complete!")
print("="*60)


RETRIEVAL VALIDATION TEST

🔍 Query: یخچال
------------------------------------------------------------

1. Product ID: oximkp
   Distance: 0.2321
   Brand: 
   Category: یخچال
   Text: یخچال فروشگاهی ۷۰ سانت جنس بدنه گالوانیزه | دسته: یخچال...

2. Product ID: njpspn
   Distance: 0.2394
   Brand: 
   Category: یخچال
   Text: یخچال مینی بار بوش مدل KUW21AHG0 | دسته: یخچال...

3. Product ID: blvked
   Distance: 0.2402
   Brand: 
   Category: یخچال
   Text: یخچال ۱۰ فوت کلاسیک امرسا | دسته: یخچال...

🔍 Query: یخچال فریزر طرح ساید پاکشوما
------------------------------------------------------------

1. Product ID: ijsjpo
   Distance: 0.2581
   Brand: 
   Category: لوازم یدکی آشپزخانه
   Text: پارچ طلقی شاهین | دسته: لوازم یدکی آشپزخانه...

2. Product ID: zoentt
   Distance: 0.2855
   Brand: 
   Category: لوازم یدکی آشپزخانه
   Text: دسته ظروف پیچ طلایی | دسته: لوازم یدکی آشپزخانه...

3. Product ID: bxrhon
   Distance: 0.2927
   Brand: 
   Category: لوازم یدکی آشپزخانه
   Text: پارچ طلقی مخ